In [25]:
# import standard python packages
import pandas as pd
import subprocess
import os
import dill
import sys
import datetime
import numpy as np

# add the utils and env directories to the path
sys.path.append('../../utils/')
sys.path.append('../../env/')

# import functions from utils directory files
from string_functions import *
from biofile_handling import *

# import paths to software installs from env
from install_locs import *

In [26]:
################
# general info #
################

species_dict = {
    'Danio_rerio': 'adultbrain',
    'Mus_musculus': 'adultbrain',
    'Xenopus_laevis': 'adultbrain'
}

global_conditions = 'adultbrain'

analysis_type = 'OrthoFinder'

################

sample_MSD = MultiSpeciesBioFileDocket(
    species_dict = species_dict,
    global_conditions = global_conditions,
    analysis_type = analysis_type
    ).unpickle()

sample_MSD.get_BioFileDockets()
sample_MSD.s3_to_local()

/home/ec2-user/glial-origins/output/DrerMmusXlae_adultbrain_OrthoFinder/ already exists
file GCF_000002035.5_GRCz10_genomic.gff already exists at /home/ec2-user/glial-origins/output/Drer_adultbrain/GCF_000002035.5_GRCz10_genomic.gff
file GCF_000002035.5_GRCz10_genomic.fna already exists at /home/ec2-user/glial-origins/output/Drer_adultbrain/GCF_000002035.5_GRCz10_genomic.fna
file GSM3768152_Brain_8_dge.txt already exists at /home/ec2-user/glial-origins/output/Drer_adultbrain/GSM3768152_Brain_8_dge.txt
file Drer_adultbrain_ZFIN_UniProtIDs.txt already exists at /home/ec2-user/glial-origins/output/Drer_adultbrain/Drer_adultbrain_ZFIN_UniProtIDs.txt
file Drer_adultbrain_uniprot-idmm.tsv already exists at /home/ec2-user/glial-origins/output/Drer_adultbrain/Drer_adultbrain_uniprot-idmm.tsv
file Drer_adultbrain_gtf-idmm.tsv already exists at /home/ec2-user/glial-origins/output/Drer_adultbrain/Drer_adultbrain_gtf-idmm.tsv
file GCF_000002035.5_GRCz10_genomic_cDNA.fna already exists at /home/ec2

In [64]:
orthogroups_file = pd.read_csv(sample_MSD.orthogroups_file.path, sep = '\t')

sp_cols = [i for i in orthogroups_file.columns if i != 'Orthogroup']

col_unpacker = lambda x: [i for i in x.split(', ')] if x is not np.nan else []

for col in sp_cols:
    orthogroups_file[col] = orthogroups_file[col].apply(col_unpacker)
    
display(orthogroups_file)

,Orthogroup,GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna.transdecoder,GCF_000002035.5_GRCz10_genomic_cDNA.fna.transdecoder,XENLA_9.2_genome_cDNA.fa.transdecoder
0,OG0000000,[],[],"[gene10395_t.p1, gene12148_t.p1, gene1307_t.p1..."
1,OG0000001,[rna75711.p2],"[gene10036.p1, gene10201.p2, gene10535.p1, gen...",[]
2,OG0000002,[],[],"[gene2928_t.p3, gene38029_t.p2, gene41218_t.p1..."
3,OG0000003,[],[],"[gene264_t.p1, rna1157.p2, rna12598.p1, rna151..."
4,OG0000004,[],[],"[gene50425_t.p1, rna27145.p2, rna34455.p1, rna..."
...,...,...,...,...
32348,OG0032348,[],[],"[rna97220.p1, rna97221.p1]"
32349,OG0032349,[],[],"[rna97260.p1, rna97260.p2]"
32350,OG0032350,[],[],"[rna97447.p1, rna97448.p1]"
32351,OG0032351,[],[],"[rna97780.p1, rna97781.p2]"


In [65]:
og_idmm_dict = {}

for sp in sample_MSD.species_BioFileDockets:
    og_idmm_dict[sp] = pd.read_csv(sample_MSD.species_BioFileDockets[sp].og_idmm.path, sep = '\t')

og_mapper_dict = {}

for sp in og_idmm_dict:
    og_mapper_dict[sp] = dict(zip(og_idmm_dict[sp]['protein_id'], og_idmm_dict[sp]['gene_name']))

In [66]:
sp_col_mapper = {'GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna.transdecoder': 'Mmus',
                'GCF_000002035.5_GRCz10_genomic_cDNA.fna.transdecoder': 'Drer',
                'XENLA_9.2_genome_cDNA.fa.transdecoder': 'Xlae'}

def dict_mapper (x, col, col_mapper, mapper_dict):
    key = col_mapper[col]
    return set([mapper_dict[key][i] for i in x])

for col in sp_cols:
    orthogroups_file[col] = orthogroups_file[col].apply(lambda x: dict_mapper(x, col, sp_col_mapper, og_mapper_dict))

display(orthogroups_file)

,Orthogroup,GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna.transdecoder,GCF_000002035.5_GRCz10_genomic_cDNA.fna.transdecoder,XENLA_9.2_genome_cDNA.fa.transdecoder
0,OG0000000,{},{},"{LOC108700417 [provisional:trim25], LOC1086994..."
1,OG0000001,{Zfp389},"{zgc:174696, LOC108183688, LOC108183294, LOC10...",{}
2,OG0000002,{},{},"{Xelaev18028992m, Xelaev18041830m, Xelaev18024..."
3,OG0000003,{},{},"{Xelaev18000208m, LOC108709124, Xelaev18022432..."
4,OG0000004,{},{},"{LOC108708057 [provisional:prkcq], LOC10871647..."
...,...,...,...,...
32348,OG0032348,{},{},{trim8.L}
32349,OG0032349,{},{},{LOC108700451 [provisional:kcnj6]}
32350,OG0032350,{},{},{gls.L}
32351,OG0032351,{},{},{vstm5.L}


In [67]:
col_repacker = lambda x: ', '.join([i for i in x]) if len(x) > 0 else np.nan

for col in sp_cols:
    orthogroups_file[col] = orthogroups_file[col].apply(col_repacker)
    
display(orthogroups_file)

,Orthogroup,GCF_000001635.23_GRCm38.p3_genomic_cDNA.fna.transdecoder,GCF_000002035.5_GRCz10_genomic_cDNA.fna.transdecoder,XENLA_9.2_genome_cDNA.fa.transdecoder
0,OG0000000,NaN,NaN,"LOC108700417 [provisional:trim25], LOC10869946..."
1,OG0000001,Zfp389,"zgc:174696, LOC108183688, LOC108183294, LOC100...",NaN
2,OG0000002,NaN,NaN,"Xelaev18028992m, Xelaev18041830m, Xelaev180246..."
3,OG0000003,NaN,NaN,"Xelaev18000208m, LOC108709124, Xelaev18022432m..."
4,OG0000004,NaN,NaN,"LOC108708057 [provisional:prkcq], LOC108716471..."
...,...,...,...,...
32348,OG0032348,NaN,NaN,trim8.L
32349,OG0032349,NaN,NaN,LOC108700451 [provisional:kcnj6]
32350,OG0032350,NaN,NaN,gls.L
32351,OG0032351,NaN,NaN,vstm5.L


In [68]:
orthogroups_file.to_csv('DrerMmusXlae_Orthogroups_bygenename.tsv', sep = '\t', index = None)